In [1]:
import numpy as np
import pandas as pd
import snap
import matplotlib.pyplot as plt
from graphviz import Digraph

In [24]:
#Load edge list into TTable
context = snap.TTableContext()
edgefilename = "./data/InvInvPairs.csv"
nodefilename = "./data/Investor.csv"
edgeschema = snap.Schema()
edgeschema.Add(snap.TStrTAttrPr("srcNID", snap.atInt))
edgeschema.Add(snap.TStrTAttrPr("dstNID", snap.atInt))
edgeschema.Add(snap.TStrTAttrPr("weight", snap.atFlt))

nodeschema = snap.Schema()
nodeschema.Add(snap.TStrTAttrPr("investorID", snap.atStr))
nodeschema.Add(snap.TStrTAttrPr("investorname", snap.atStr))

edge_table = snap.TTable.LoadSS(edgeschema, edgefilename, context, "\t", snap.TBool(False))
node_table = snap.TTable.LoadSS(nodeschema, nodefilename, context, "\t", snap.TBool(False))

In [25]:
#Convert TTable into Directed graph
edgeattrv = snap.TStrV()
edgeattrv.Add("weight")

nodeattrv = snap.TStrV()
nodeattrv.Add("investorname")
net = snap.ToNetwork(snap.PNEANet, edge_table, "srcNID", "dstNID", edgeattrv, node_table, "investorID", nodeattrv, snap.aaFirst)

In [26]:
snap.PrintInfo(net, "Reduced Investor-Investor DGraph Stats", "./data/ReducedInvDGraph-info.txt", False)

In [27]:
#In-edge subgraph for Berkshire
investorname = "BERKSHIRE HATHAWAY INC"
id = SF3_investorsID[investorname]
dot = Digraph(comment="In-edge egonet of {}".format(investorname), engine='sfdp', node_attr={'shape':'oval'}, format='png')

list_node = [id]
list_edge = []
list_nbrnbr = []
dot.node(SF3_investors[id], color = 'red', fontcolor = 'red')
Node = net.GetNI(id)

k = Node.GetInDeg()
for i in range(k):
    In = Node.GetInNId(i)
    if In not in list_node:
        dot.node(SF3_investors[In])
        list_node += [In]
    if ([In, id]) not in list_edge:
        dot.edge(SF3_investors[In], SF3_investors[id])
        list_edge += [([In, id])]
    InNode = net.GetNI(In)
    
    k_In = InNode.GetInDeg()
    for j in range(k_In):
        InIn = InNode.GetInNId(j)
        list_nbrnbr += [InIn]
        if InIn not in list_node:
            dot.node(SF3_investors[InIn])
            list_node += [InIn]
        if ([InIn, In]) not in list_edge:
            dot.edge(SF3_investors[InIn], SF3_investors[In])
            list_edge += [([InIn, In])]
            
list_nbrnbr = list(set(list_nbrnbr))
for i in range(len(list_nbrnbr)-1):
    for j in range(i, len(list_nbrnbr)):
        if net.IsEdge(list_nbrnbr[i], list_nbrnbr[j]):
            if [list_nbrnbr[i], list_nbrnbr[j]] not in list_edge:
                dot.edge(SF3_investors[list_nbrnbr[i]], SF3_investors[list_nbrnbr[j]])                
        if net.IsEdge(list_nbrnbr[j], list_nbrnbr[i]):
            if [list_nbrnbr[j], list_nbrnbr[i]] not in list_edge:
                dot.edge(SF3_investors[list_nbrnbr[j]], SF3_investors[list_nbrnbr[i]])

dot.render('./data/In_edge_subgraph_node{}'.format(id), view=True)

'In_edge_subgraph_node648.png'

In [28]:
#Out-edge subgraph for Berkshire
investorname = "BERKSHIRE HATHAWAY INC"
id = SF3_investorsID[investorname]
dot = Digraph(comment="Out-edge egonet of {}".format(investorname), engine='sfdp', node_attr={'shape':'oval'}, format='png')

list_node = [id]
list_edge = []
list_nbrnbr = []
dot.node(SF3_investors[id], color = 'red', fontcolor = 'red')
Node = net.GetNI(id)

k = Node.GetOutDeg()
for i in range(k):
    Out = Node.GetOutNId(i)
    if Out not in list_node:
        dot.node(SF3_investors[Out])
        list_node += [Out]
    if ([id, Out]) not in list_edge:
        dot.edge(SF3_investors[id], SF3_investors[Out])
        list_edge += [([id, Out])]
    OutNode = net.GetNI(Out)
    
    k_Out = OutNode.GetOutDeg()
    for j in range(k_Out):
        OutOut = OutNode.GetOutNId(j)
        list_nbrnbr += [OutOut]
        if OutOut not in list_node:
            dot.node(SF3_investors[OutOut])
            list_node += [OutOut]
        if ([Out, OutOut]) not in list_edge:
            dot.edge(SF3_investors[Out], SF3_investors[OutOut])
            list_edge += [([Out, OutOut])]
        
list_nbrnbr = list(set(list_nbrnbr))
for i in range(len(list_nbrnbr)-1):
    for j in range(i, len(list_nbrnbr)):
        if net.IsEdge(list_nbrnbr[i], list_nbrnbr[j]):
            if [list_nbrnbr[i], list_nbrnbr[j]] not in list_edge:
                dot.edge(SF3_investors[list_nbrnbr[i]], SF3_investors[list_nbrnbr[j]])                
        if net.IsEdge(list_nbrnbr[j], list_nbrnbr[i]):
            if [list_nbrnbr[j], list_nbrnbr[i]] not in list_edge:
                dot.edge(SF3_investors[list_nbrnbr[j]], SF3_investors[list_nbrnbr[i]])

dot.render('./data/Out_edge_subgraph_node{}'.format(id), view=True)

'Out_edge_subgraph_node648.png'